In [1]:
!pip install -r requirements.txt

In [2]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
# Importing the dataset
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values

In [4]:
# Encoding categorical data
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])
onehotencoder = OneHotEncoder(categorical_features = [1])
X = onehotencoder.fit_transform(X).toarray()
X = X[:, 1:]

/Users/gabrielfilipe7/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/Users/gabrielfilipe7/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:392: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


In [5]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [6]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [7]:
# Part 2 - Now let's make the ANN!

# Importing the Keras libraries and packages
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

Using TensorFlow backend.


In [8]:
# Initialising the ANN
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))

# Adding the second hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))

# Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


In [9]:
# Fitting the ANN to the Training set
classifier.fit(X_train, y_train, batch_size = 10, epochs = 10)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
8000/8000 [==============================] - 1s 137us/step - loss: 0.4808 - acc: 0.7957
Epoch 2/10
8000/8000 [==============================] - 1s 118us/step - loss: 0.4290 - acc: 0.7960
Epoch 3/10
8000/8000 [==============================] - 1s 108us/step - loss: 0.4232 - acc: 0.7960
Epoch 4/10
8000/8000 [==============================] - 1s 109us/step - loss: 0.4190 - acc: 0.8175
Epoch 5/10
8000/8000 [==============================] - 1s 114us/step - loss: 0.4162 - acc: 0.8259
Epoch 6/10
8000/8000 [==============================] - 1s 100us/step - loss: 0.4142 - acc: 0.8271
Epoch 7/10
8000/8000 [==============================] - 1s 99us/step - loss: 0.4126 - acc: 0.8310
Epoch 8/10
8000/8000 [==============================] - 1s 98us/step - loss: 0.4111 - acc: 0.8327
Epoch 9/10
8000/8000 [==============================] - 1s 97us/step - loss: 0.4099 - acc: 0.8306
Epoch 10/10
8000/8000 [==============================] - 1s 102u

In [10]:
# Part 3 - Making predictions and evaluating the model

# Predicting the Test set results
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [11]:
# Predicting a single new observation
"""Predict if the customer with the following informations will leave the bank:
Geography: France
Credit Score: 600
Gender: Male
Age: 40
Tenure: 3
Balance: 60000
Number of Products: 2
Has Credit Card: Yes
Is Active Member: Yes
Estimated Salary: 50000"""
new_prediction = classifier.predict(sc.transform(np.array([[0.0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])))
new_prediction = (new_prediction > 0.5)

In [12]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [13]:
# Part 4 - Evaluating, Improving and Tuning the ANN

# Evaluating the ANN
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense
#from keras.layers import Dropout
def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 600, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 50, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 10)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
mean = accuracies.mean()
variance = accuracies.std()

In [14]:
# Improving the ANN
# Dropout Regularization to reduce overfitting if needed

# Tuning the ANN
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense

def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier)

In [15]:
parameters = {'batch_size': [15, 20, 10],
              'epochs': [10, 20, 10],
              'optimizer': ['adam', 'rmsprop']}

grid_search = GridSearchCV(estimator = classifier,
                           scoring = 'accuracy',
                           param_grid = parameters,
                           cv = 10)

In [16]:
grid_search = grid_search.fit(X_train, y_train)

best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Epoch 1/10
7200/7200 [==============================] - 1s 112us/step - loss: 0.5085 - acc: 0.7965
Epoch 2/10
7200/7200 [==============================] - 1s 72us/step - loss: 0.4313 - acc: 0.8108
Epoch 3/10
7200/7200 [==============================] - 1s 77us/step - loss: 0.4193 - acc: 0.8200
Epoch 4/10
7200/7200 [==============================] - 1s 73us/step - loss: 0.4138 - acc: 0.8183
Epoch 5/10
7200/7200 [==============================] - 1s 74us/step - loss: 0.4083 - acc: 0.8199
Epoch 6/10
7200/7200 [==============================] - 1s 75us/step - loss: 0.4062 - acc: 0.8228
Epoch 7/10
7200/7200 [==============================] - 1s 73us/step - loss: 0.4036 - acc: 0.8240
Epoch 8/10
7200/7200 [==============================] - 1s 73us/step - loss: 0.4068 - acc: 0.8222
Epoch 9/10
7200/7200 [==============================] - 1s 73us/step - loss: 0.4029 - acc: 0.8229
Ep

7200/7200 [==============================] - 1s 76us/step - loss: 0.4091 - acc: 0.8196
Epoch 1/10
7200/7200 [==============================] - 1s 176us/step - loss: 0.5347 - acc: 0.7951
Epoch 2/10
7200/7200 [==============================] - 1s 79us/step - loss: 0.4356 - acc: 0.7957
Epoch 3/10
7200/7200 [==============================] - 1s 80us/step - loss: 0.4327 - acc: 0.7957
Epoch 4/10
7200/7200 [==============================] - 1s 77us/step - loss: 0.4281 - acc: 0.7957
Epoch 5/10
7200/7200 [==============================] - 1s 80us/step - loss: 0.4251 - acc: 0.7957
Epoch 6/10
7200/7200 [==============================] - 1s 80us/step - loss: 0.4225 - acc: 0.7957
Epoch 7/10
7200/7200 [==============================] - 1s 79us/step - loss: 0.4160 - acc: 0.7957
Epoch 8/10
7200/7200 [==============================] - 1s 79us/step - loss: 0.4159 - acc: 0.7988
Epoch 9/10
7200/7200 [==============================] - 1s 78us/step - loss: 0.4119 - acc: 0.8235
Epoch 10/10
7200/7200 [=======

7200/7200 [==============================] - 1s 208us/step - loss: 0.5275 - acc: 0.7968
Epoch 2/10
7200/7200 [==============================] - 1s 79us/step - loss: 0.4387 - acc: 0.7969
Epoch 3/10
7200/7200 [==============================] - 1s 81us/step - loss: 0.4265 - acc: 0.7978
Epoch 4/10
7200/7200 [==============================] - 1s 87us/step - loss: 0.4161 - acc: 0.8078
Epoch 5/10
7200/7200 [==============================] - 1s 79us/step - loss: 0.4089 - acc: 0.8121
Epoch 6/10
7200/7200 [==============================] - 1s 73us/step - loss: 0.3982 - acc: 0.8281
Epoch 7/10
7200/7200 [==============================] - 1s 71us/step - loss: 0.3952 - acc: 0.8364
Epoch 8/10
7200/7200 [==============================] - 1s 72us/step - loss: 0.3858 - acc: 0.8435
Epoch 9/10
7200/7200 [==============================] - 1s 80us/step - loss: 0.3783 - acc: 0.8454
Epoch 10/10
7200/7200 [==============================] - 1s 78us/step - loss: 0.3837 - acc: 0.8479
Epoch 1/10
7200/7200 [=======

7200/7200 [==============================] - 1s 84us/step - loss: 0.4368 - acc: 0.7956
Epoch 3/20
7200/7200 [==============================] - 1s 83us/step - loss: 0.4338 - acc: 0.7956: 0s - loss: 0.4365 - acc: 0.79
Epoch 4/20
7200/7200 [==============================] - 1s 84us/step - loss: 0.4315 - acc: 0.7956
Epoch 5/20
7200/7200 [==============================] - 1s 82us/step - loss: 0.4274 - acc: 0.7956
Epoch 6/20
7200/7200 [==============================] - 1s 81us/step - loss: 0.4301 - acc: 0.7956
Epoch 7/20
7200/7200 [==============================] - 1s 82us/step - loss: 0.4253 - acc: 0.8174
Epoch 8/20
7200/7200 [==============================] - 1s 81us/step - loss: 0.4264 - acc: 0.8240: 0s - loss: 0.4262 - acc:
Epoch 9/20
7200/7200 [==============================] - 1s 82us/step - loss: 0.4278 - acc: 0.8214
Epoch 10/20
7200/7200 [==============================] - 1s 81us/step - loss: 0.4256 - acc: 0.8254
Epoch 11/20
7200/7200 [==============================] - 1s 82us/step -

7200/7200 [==============================] - 2s 278us/step - loss: 0.5076 - acc: 0.7969
Epoch 2/20
7200/7200 [==============================] - 1s 90us/step - loss: 0.4366 - acc: 0.7969
Epoch 3/20
7200/7200 [==============================] - 1s 90us/step - loss: 0.4338 - acc: 0.7969
Epoch 4/20
7200/7200 [==============================] - 1s 91us/step - loss: 0.4306 - acc: 0.7969: 0s - loss: 0.4268 - acc: 0.8
Epoch 5/20
7200/7200 [==============================] - 1s 90us/step - loss: 0.4289 - acc: 0.7969
Epoch 6/20
7200/7200 [==============================] - 1s 90us/step - loss: 0.4302 - acc: 0.7969
Epoch 7/20
7200/7200 [==============================] - 1s 91us/step - loss: 0.4247 - acc: 0.7969
Epoch 8/20
7200/7200 [==============================] - 1s 91us/step - loss: 0.4258 - acc: 0.8114
Epoch 9/20
7200/7200 [==============================] - 1s 90us/step - loss: 0.4253 - acc: 0.8200
Epoch 10/20
7200/7200 [==============================] - 1s 91us/step - loss: 0.4292 - acc: 0.8226

...

7200/7200 [==============================] - 2s 338us/step - loss: 0.4267 - acc: 0.8267
Epoch 14/20
7200/7200 [==============================] - 3s 365us/step - loss: 0.4235 - acc: 0.8315
Epoch 15/20
7200/7200 [==============================] - 2s 346us/step - loss: 0.4254 - acc: 0.8296
Epoch 16/20
7200/7200 [==============================] - 3s 354us/step - loss: 0.4265 - acc: 0.8294
Epoch 17/20
7200/7200 [==============================] - 2s 345us/step - loss: 0.4283 - acc: 0.8296
Epoch 18/20
7200/7200 [==============================] - 2s 331us/step - loss: 0.4248 - acc: 0.8308
Epoch 19/20
7200/7200 [==============================] - 3s 349us/step - loss: 0.4210 - acc: 0.8306
Epoch 20/20
7200/7200 [==============================] - 3s 377us/step - loss: 0.4251 - acc: 0.8285
Epoch 1/20
7200/7200 [==============================] - 10s 1ms/step - loss: 0.5109 - acc: 0.7956
Epoch 2/20
7200/7200 [==============================] - 2s 318us/step - loss: 0.4394 - acc: 0.7956
Epoch 3/20
7200

7200/7200 [==============================] - 3s 387us/step - loss: 0.4319 - acc: 0.8193
Epoch 16/20
7200/7200 [==============================] - 3s 382us/step - loss: 0.4333 - acc: 0.8187
Epoch 17/20
7200/7200 [==============================] - 2s 339us/step - loss: 0.4335 - acc: 0.8175
Epoch 18/20
7200/7200 [==============================] - 2s 338us/step - loss: 0.4318 - acc: 0.8204
Epoch 19/20
7200/7200 [==============================] - 3s 380us/step - loss: 0.4350 - acc: 0.8246
Epoch 20/20
7200/7200 [==============================] - 3s 377us/step - loss: 0.4313 - acc: 0.8217
Epoch 1/20
7200/7200 [==============================] - 11s 1ms/step - loss: 0.5131 - acc: 0.7965
Epoch 2/20
7200/7200 [==============================] - 3s 383us/step - loss: 0.4397 - acc: 0.7969
Epoch 3/20
7200/7200 [==============================] - 2s 334us/step - loss: 0.4354 - acc: 0.7969
Epoch 4/20
7200/7200 [==============================] - 3s 367us/step - loss: 0.4318 - acc: 0.7969
Epoch 5/20
7200/7

7200/7200 [==============================] - 3s 441us/step - loss: 0.4097 - acc: 0.8307
Epoch 18/20
7200/7200 [==============================] - 3s 369us/step - loss: 0.4109 - acc: 0.8275
Epoch 19/20
7200/7200 [==============================] - 3s 351us/step - loss: 0.4100 - acc: 0.8335
Epoch 20/20
7200/7200 [==============================] - 3s 372us/step - loss: 0.4106 - acc: 0.8353
Epoch 1/10
7200/7200 [==============================] - 11s 2ms/step - loss: 0.4976 - acc: 0.7965: 8s - l
Epoch 2/10
7200/7200 [==============================] - 3s 396us/step - loss: 0.4390 - acc: 0.7971
Epoch 3/10
7200/7200 [==============================] - 3s 427us/step - loss: 0.4338 - acc: 0.7971
Epoch 4/10
7200/7200 [==============================] - 3s 402us/step - loss: 0.4307 - acc: 0.7971
Epoch 5/10
7200/7200 [==============================] - 3s 406us/step - loss: 0.4298 - acc: 0.7971
Epoch 6/10
7200/7200 [==============================] - 3s 379us/step - loss: 0.4285 - acc: 0.7971
Epoch 7/10


7200/7200 [==============================] - 3s 370us/step - loss: 0.4237 - acc: 0.8265
Epoch 10/10
7200/7200 [==============================] - 3s 370us/step - loss: 0.4211 - acc: 0.8268
Epoch 1/10
7200/7200 [==============================] - 11s 2ms/step - loss: 0.4983 - acc: 0.7950
Epoch 2/10
7200/7200 [==============================] - 3s 405us/step - loss: 0.4329 - acc: 0.7957
Epoch 3/10
7200/7200 [==============================] - 3s 414us/step - loss: 0.4275 - acc: 0.7957
Epoch 4/10
7200/7200 [==============================] - 3s 357us/step - loss: 0.4284 - acc: 0.7957
Epoch 5/10
7200/7200 [==============================] - 3s 406us/step - loss: 0.4266 - acc: 0.7957
Epoch 6/10
7200/7200 [==============================] - 3s 430us/step - loss: 0.4251 - acc: 0.8065
Epoch 7/10
7200/7200 [==============================] - 3s 443us/step - loss: 0.4209 - acc: 0.8233
Epoch 8/10
7200/7200 [==============================] - 3s 422us/step - loss: 0.4238 - acc: 0.8239
Epoch 9/10
7200/7200 

7200/7200 [==============================] - 13s 2ms/step - loss: 0.5183 - acc: 0.7968
Epoch 2/10
7200/7200 [==============================] - 3s 398us/step - loss: 0.4348 - acc: 0.7969
Epoch 3/10
7200/7200 [==============================] - 3s 411us/step - loss: 0.4285 - acc: 0.7969
Epoch 4/10
7200/7200 [==============================] - 4s 491us/step - loss: 0.4230 - acc: 0.7969
Epoch 5/10
7200/7200 [==============================] - 3s 434us/step - loss: 0.4236 - acc: 0.7969
Epoch 6/10
7200/7200 [==============================] - 3s 432us/step - loss: 0.4173 - acc: 0.7997
Epoch 7/10
7200/7200 [==============================] - 3s 445us/step - loss: 0.4123 - acc: 0.8222
Epoch 8/10
7200/7200 [==============================] - 3s 418us/step - loss: 0.4142 - acc: 0.8210
Epoch 9/10
7200/7200 [==============================] - 3s 467us/step - loss: 0.4139 - acc: 0.8200
Epoch 10/10
7200/7200 [==============================] - 3s 481us/step - loss: 0.4086 - acc: 0.8271
Epoch 1/10
7200/7200 

## Montando um DataFrame com os dados resultados

In [27]:
dados = {'Melhor Acurácia':best_accuracy, 'Melhores parâmetros' : best_parameters,'Média': mean,'Variância' : variance}

In [40]:
dados_finais = pd.DataFrame.from_dict(dados, orient='index',columns=['Dados'])

In [41]:
dados_finais

,Dados
Melhor Acurácia,0.8345
Melhores parâmetros,"{'batch_size': 10, 'epochs': 20, 'optimizer': ..."
Média,0.85825
Variância,0.0121089
